<a href="https://colab.research.google.com/github/JoyeBright/Multilingual_Multimodal_Universal_MT/blob/main/OpenNMT_GRU_ManythingsDataset_Per_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install OpenNMT**

In [ ]:
!pip install --upgrade pip
!pip install OpenNMT-tf

     |████████████████████████████████| 1.5MB 20.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 145 kB 21.6 MB/s 
     |████████████████████████████████| 65 kB 5.2 MB/s 
     |████████████████████████████████| 75.5 MB 18 kB/s 
     |████████████████████████████████| 269 kB 84.0 MB/s 
     |████████████████████████████████| 703 kB 58.0 MB/s 
     |████████████████████████████████| 2.5 MB 45.6 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=f89031ed89b2789faa9877a839f23cfc65c6f9f96e6b717a3ac8a67fb5fddfe2
  Stored in directory: /root/.cache/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80fc
Successfully built pyyaml
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.

### **OpenNMT version**

In [ ]:
!onmt-main -v

2021-01-25 09:43:22.488662: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
OpenNMT-tf 2.14.0


### **Preparing Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget "https://www.manythings.org/anki/pes-eng.zip"

--2021-01-23 17:06:03--  https://www.manythings.org/anki/pes-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3036::ac43:adc6, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.55.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112553 (110K) [application/zip]
Saving to: ‘pes-eng.zip’

pes-eng.zip         100%[===================>] 109.92K   527KB/s    in 0.2s    

2021-01-23 17:06:04 (527 KB/s) - ‘pes-eng.zip’ saved [112553/112553]



In [ ]:
!unzip pes-eng.zip -d "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data"

Archive:  pes-eng.zip
  inflating: /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/_about.txt  
  inflating: /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/pes.txt  


In [ ]:
path = "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/pes.txt"

In [ ]:
f = open(path, "r")
print(f.read(978)) 

Who?	چه کسی؟	CC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #4746184 (mahdiye)
Go on.	ادامه بده ( ادامه دادن )	CC-BY 2.0 (France) Attribution: tatoeba.org #2230774 (CK) & #6669169 (arashorosia)
Smile.	لبخند بزن.	CC-BY 2.0 (France) Attribution: tatoeba.org #2764108 (CK) & #4746196 (mahdiye)
Attack!	حمله!	CC-BY 2.0 (France) Attribution: tatoeba.org #1972610 (CK) & #4746181 (mahdiye)
Got it!	گرفتم!	CC-BY 2.0 (France) Attribution: tatoeba.org #320484 (CM) & #888645 (mahdiye)
I know.	من می دانم.	CC-BY 2.0 (France) Attribution: tatoeba.org #319990 (CK) & #888643 (mahdiye)
Listen.	گوش کن.	CC-BY 2.0 (France) Attribution: tatoeba.org #1913088 (CK) & #4746178 (mahdiye)
Really?	واقعا؟	CC-BY 2.0 (France) Attribution: tatoeba.org #373216 (kotobaboke) & #5118169 (Hussein64)
Really?	جدا؟	CC-BY 2.0 (France) Attribution: tatoeba.org #373216 (kotobaboke) & #5118171 (Hussein64)
Why me?	چرا من؟	CC-BY 2.0 (France) Attribution: tatoeba.org #24958 (CK) & #4746261 (mahdiye)



In [ ]:
import re

In [ ]:
text_file = open("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/English.txt", "w")
text_file2 = open("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/Persian.txt", "w")

f = open(path, "r")
for r in f:
  text_file.write(re.split(r'\t+', r)[0]+"\n") # English txt
  text_file2.write(re.split(r'\t+', r)[1]+"\n") # Persian txt

In [ ]:
text_file.close()
text_file2.close()

### **Dataset Splits: Train, Validation, Test**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [ ]:
English = np.genfromtxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/English.txt", dtype='str', delimiter="\n")
Persian = np.genfromtxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/Persian.txt", dtype='str', delimiter="\n")

In [ ]:
print(English[:10]) #target
print(Persian[:10]) #source

['Who?' 'Go on.' 'Smile.' 'Attack!' 'Got it!' 'I know.' 'Listen.'
 'Really?' 'Really?' 'Why me?']
['چه کسی؟' 'ادامه بده ( ادامه دادن )' 'لبخند بزن.' 'حمله!' 'گرفتم!'
 'من می دانم.' 'گوش کن.' 'واقعا؟' 'جدا؟' 'چرا من؟']


In [ ]:
src_train, src_test, tgt_train, tgt_test = train_test_split(Persian, English, test_size=0.2, random_state=1)

src_train, src_val, tgt_train, tgt_val = train_test_split(src_train, tgt_train, test_size=0.25, random_state=1)

In [ ]:
print("Source (Train, Test, Val):" , src_train.shape[0], src_test.shape[0], src_val.shape[0])
print("Target (Train, Test, Va)l:" , tgt_train.shape[0], tgt_test.shape[0], tgt_val.shape[0])

Source (Train, Test, Val): 1365 455 455
Target (Train, Test, Va)l: 1365 455 455


In [ ]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-train.txt", src_train, encoding="UTF-8", fmt = '%s')

In [ ]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-train.txt", tgt_train, encoding="UTF-8", fmt = '%s')

In [ ]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-test.txt", src_test, encoding="UTF-8", fmt = '%s')

In [ ]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-test.txt", tgt_test, encoding="UTF-8", fmt = '%s')

In [ ]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-val.txt", src_val, encoding="UTF-8", fmt = '%s')

In [ ]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-val.txt", tgt_val, encoding="UTF-8", fmt = '%s')

### **Creating Vocabulary**

In [ ]:
!pip install sacremoses # used as an English tokenizer

     |████████████████████████████████| 883 kB 19.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893258 sha256=56cf0b20cbbaf9aed06ea992b72194d765c9cf088b6dde511dd113ceaac25fac
  Stored in directory: /root/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses


### **Generate English vocabulary from raw training files with on-the-fly tokenization (online)**

In [ ]:
!onmt-build-vocab --size 50000 --save_vocab /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/data/tgt-vocab.txt /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-train.txt

2021-01-23 21:17:37.995452: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-23 21:17:40.292901: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-23 21:17:40.293820: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-23 21:17:40.304732: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-01-23 21:17:40.304789: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dced393d028e): /proc/driver/nvidia/version does not exist
2021-01-23 21:17:40.305329: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


### **Generate Persian vocabulary from customized tokenization**

In [ ]:
from __future__ import unicode_literals
!pip install hazm
from hazm import *

In [ ]:
def preprocess_persian_sentence(w):
  normalizer = Normalizer() # Persian normalizer instance (Hazm)
  w = normalizer.normalize(w)
  w = word_tokenize(w)
  return w

In [ ]:
import io

text_file = "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-train.txt"

tokenized_file = "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/test.txt"

lines = io.open(text_file, encoding="UTF-8").read()

tokenized_array = []

for line in lines.splitlines():
  # print(preprocess_persian_sentence(line))
  # print(type(preprocess_persian_sentence(line)))
  print(" ".join(str(x) for x in preprocess_persian_sentence(line)))
  tokenized_array.append(" ".join(str(x) for x in preprocess_persian_sentence(line)))

In [ ]:
# print(tokenized_array)
with open(tokenized_file, 'w') as f:
    for item in tokenized_array:
        f.write("%s\n" % item)

**Generating vocabulary from Persian tokenized sentences**

In [ ]:
!onmt-build-vocab --size 50000 --save_vocab /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/data/src-vocab.txt /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-test.txt

2021-01-23 21:17:11.832052: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-23 21:17:14.133504: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-23 21:17:14.134482: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-23 21:17:14.145235: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-01-23 21:17:14.145300: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dced393d028e): /proc/driver/nvidia/version does not exist
2021-01-23 21:17:14.145824: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


### **Model Training**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
!onmt-main -h

2021-01-25 10:07:20.291675: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
usage: onmt-main [-h] [-v] --config CONFIG [CONFIG ...] [--auto_config]
                 [--model_type {GPT2Small,ListenAttendSpell,LstmCnnCrfTagger,LuongAttention,NMTBigV1,NMTMediumV1,NMTSmallV1,Transformer,TransformerBase,TransformerBaseRelative,TransformerBig,TransformerBigRelative,TransformerRelative}]
                 [--model MODEL] [--run_dir RUN_DIR] [--data_dir DATA_DIR]
                 [--checkpoint_path CHECKPOINT_PATH]
                 [--log_level {CRITICAL,ERROR,WARNING,INFO,DEBUG,NOTSET}]
                 [--seed SEED] [--gpu_allow_growth]
                 [--intra_op_parallelism_threads INTRA_OP_PARALLELISM_THREADS]
                 [--inter_op_parallelism_threads INTER_OP_PARALLELISM_THREADS]
                 [--mixed_precision] [--eager_execution]
                 {train,eval,infer,export,score,average_checkpoints,update_vo

In [15]:
!onmt-main --model_type LuongAttention --config /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/config/config_train_rnn.yml --auto_config train --with_eval

2021-01-25 10:33:33.767138: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-25 10:33:35.493717: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-25 10:33:35.494787: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-25 10:33:35.551857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-25 10:33:35.552417: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2021-01-25 10:33:35.552453: I tensorflow/stream_executor/platform/default/dso_loade